In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

# Intro to FITS Files

In [2]:
butterfly_nebula = "butterfly_nebula_hubble.fits.gz" #path to file 

In [3]:
fits.info(butterfly_nebula) #gives you information about the fits file 

Filename: butterfly_nebula_hubble.fits.gz
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     580   (5600, 4200, 3)   float32   


In [4]:
#acessing the header 
hdu = fits.open(butterfly_nebula) # opening a fits file 
header = hdu[0].header # header of data stored in No. 0 (the primary HDU)
print(header)
hdu.close() #closing a fits file 

SIMPLE  =                    T / file does conform to FITS standard             BITPIX  =                  -32 / number of bits per data pixel                  NAXIS   =                    3 / number of data axes                            NAXIS1  =                 5600 / length of data axis 1                          NAXIS2  =                 4200 / length of data axis 2                          NAXIS3  =                    3 / length of data axis 3                          EXTEND  =                    T / FITS dataset may contain extensions            COMMENT   FITS (Flexible Image Transport System) format is defined in 'AstronomyCOMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H DATE    = '2025-05-05T03:27:58' / file creation date (YYYY-MM-DDThh:mm:ss UT)   OBJECT  = 'NGC 6302 in S II'   / Name of the object observed                    ORIGIN  = 'NOAO-IRAF FITS Image Kernel July 2003' / FITS file originator        IRAF-TLM= '17:16:44 (03/09/2009)' / Time

In [5]:
#acessing information stored in keywords of the header 
print(header["RA_TARG"]) 
print(header["DEC_TARG"]) 

258.4312254167
-37.10384166667


In [6]:
# acessing data stored within a fits file 
hdu = fits.open(butterfly_nebula)
data = hdu[0].data #data stored in No. 0 (the primary HDU)
print(data.shape) # 3 filters stored as 3D cube 
filter_1 = data[0,:,:] #getting the first filter, data[1,:,:] holds the second filter, data[2,:,:] holds the third filter 
hdu.close()

(3, 4200, 5600)


In [ ]:
from astropy.visualization import ImageNormalize, PercentileInterval, SqrtStretch, LogStretch # functions to help with data visualization 

In [ ]:
norm = ImageNormalize(stretch=SqrtStretch(), vmin=0, vmax=0.4) #defines a square root stretch that we can apply to the image, sets the minimum and maximum pixel values to show

plt.figure(figsize=(10, 8))
plt.imshow(filter_1, cmap='viridis', origin='lower', norm=norm) #plot 2d data with normalization defined by norm, cmap sets the colormap to use, origin='lower' gets you pixel (0,0) in the lower left corner
plt.colorbar()
plt.title("Butterfly Nebula")
plt.show()

In [ ]:
print("Let's print the mean and median value in filter 1!")
print("Mean=", np.mean(filter_1))
print("Median=", np.median(filter_1))

In [ ]:
print("Let's try the functions that ignore NaNs:")
print("Mean=", np.nanmean(filter_1))
print("Median=", np.nanmedian(filter_1))

In [ ]:
print("Maximum Pixel Value=", np.nanmax(filter_1)) 
print("Minimum Pixel Value=", np.nanmin(filter_1))

In [ ]:
filter1_data_without_NaNs = filter_1[0:3800,900:4600] # slice to get data free from NaNs (use plotted image to guide the range of indices)

mean_of_each_col = np.nanmedian(filter1_data_without_NaNs, axis=0) #Find the Median across each column (axis=1 gives the median across each row)

In [ ]:
plt.figure(figsize=(8, 5))
plt.hist(mean_of_each_col, bins=50, color="dodgerblue", alpha=0.8) #plotting a histogram of the array of medians where bins=50 gives 50 total bins in our histogram

plt.xlabel("Pixel Intensity")
plt.ylabel("Frequency")
plt.title("Histogram of Column Medians")
plt.yscale('log')  # log scale to see low-frequency bins

plt.show()

# FITS File Excercise

Extract the Target Name, Sun Angle and Moon Angle from the header.

Open the fits file as above and store data[1,:,:] and data[2,:,:] as variables named filter_2 and filter_3 respectively.

Plot filter_2 and filter_3 using stretches that showcase features of the butterfly nebula.

Extract the mean and median of the images. Find the maximum and minimum pixel values in each filter. Then, make a histogram that shows the distribution of the median of each row in filter_2 and filter_3.

You can combine the three filters to make an RGB image (this is how we get all of the pretty color images!)

In [ ]:
from astropy.visualization import make_lupton_rgb

In [ ]:
# Create RGB image (can adjust stretch & Q for contrast)
interval = PercentileInterval(99.5)  # Clip top/bottom 0.25% and keep 99.5% of the original data; this can be applied to our filters 
filter_2 = data[1,:,:] #equivalent to what you did above! 
filter_3 = data[2,:,:] #equivalent to what you did above! 

rgb_image = make_lupton_rgb(interval(filter_1), # your reddest filter (shows up as red in final image)
                            interval(filter_2), # your intermediate filter (shows up as green in final image)
                            interval(filter_3), # your bluest filter (shows up as blue in final image)
                            Q=6, stretch=0.1)   #parameters that adjust the stretch to make features visible 

plt.figure(figsize=(10, 10))
plt.imshow(rgb_image, origin='lower')
plt.axis('off') #removes the axes with the pixel numbers labeled
plt.title("RGB Composite")
plt.show()